# KE5006 Applied Research

### Identifying enhancers and their strength with deep neural networks

# Preliminary Layer 1 Model

## Summary of Findings
* to be updated


## Load libraries

In [1]:
# Set the working directory (which contains the directories source, data, etc.)
import os
os.chdir(os.path.join(os.path.sep, 'home', 'tkokkeng', 'Documents', 'KE5006-AppliedResearch', 'enhancer'))
os.getcwd()

'/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer'

In [2]:
# Check if the directory containing the source files are in the path.
import sys
if os.path.join(os.getcwd(), 'source') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'source'))
sys.path

['/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer',
 '',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages',
 '/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer/source',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python36.zip',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/tkokkeng/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/IPython/extensions',
 '/home/tkokkeng/.ipython']

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import pickle

import myUtilities as mu

from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, make_scorer, recall_score, roc_auc_score, roc_curve 
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, GridSearchCV


from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


## Load data

In [4]:
enhancer_df = pd.read_csv(os.path.join('data', 'enhancer.csv'))
enhancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 2 columns):
id          1484 non-null object
sequence    1484 non-null object
dtypes: object(2)
memory usage: 23.3+ KB


In [5]:
enhancer_df['enhancer'] = np.ones((len(enhancer_df),))

In [6]:
enhancer_df.head()

,id,sequence,enhancer
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


In [7]:
non_enhancer_df = pd.read_csv(os.path.join('data', 'non_enhancer.csv'))
non_enhancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 2 columns):
id          1484 non-null object
sequence    1484 non-null object
dtypes: object(2)
memory usage: 23.3+ KB


In [8]:
non_enhancer_df['enhancer'] = np.zeros((len(non_enhancer_df),))

In [9]:
non_enhancer_df.head()

,id,sequence,enhancer
0,CHRX_2970600_2970800,CAGTCACATCTGTAATCACAATACGTTGGGAGGCTGAGGCAGGAGG...,0.0
1,CHRX_6179400_6179600,ACTTTGAAGAAGTCAGTCATCAAGATGAGAGACCCAACTGTCAAGC...,0.0
2,CHRX_11003079_11003279,TCGGCCTCCCAAAGTGCTGGGATTATAGGCATGAGCTACTGCACCC...,0.0
3,CHRX_22042679_22042879,TGGGAGCTGTATCAATCATGTTTTTTATTTTCTATATTTTATGATG...,0.0
4,CHRX_23280479_23280679,TACAGCAAATAGCCTTGGCAGATACAGTGTTTCCCTCCAGAGCAAA...,0.0


## Combine the data frames to form a single dataset

In [10]:
all_data_df = pd.concat([enhancer_df, non_enhancer_df])
all_data_df.reset_index(drop=True, inplace=True)
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2968 entries, 0 to 2967
Data columns (total 3 columns):
id          2968 non-null object
sequence    2968 non-null object
enhancer    2968 non-null float64
dtypes: float64(1), object(2)
memory usage: 69.6+ KB


In [11]:
all_data_df.head()

,id,sequence,enhancer
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


All the sequences are of length 200 characters.

In [12]:
all_data_df['sequence'].map(lambda x: len(x)).value_counts()

200    2968
Name: sequence, dtype: int64

## Prepare the sequence data for modelling

Initialise the keras tokenizer

In [13]:
tokenizer =  Tokenizer(num_words=4, lower=False, char_level=True)

In [14]:
tokenizer.fit_on_texts(all_data_df['sequence'][0])

In [15]:
tokenizer.word_index

{'G': 1, 'T': 2, 'A': 3, 'C': 4}

In [16]:
tokenizer.index_word

{1: 'G', 2: 'T', 3: 'A', 4: 'C'}

Test the tokenizer on the first sequence

In [17]:
all_data_df['sequence'][0][:10]

'CACAATGTAG'

In [18]:
tokenizer.texts_to_matrix(all_data_df['sequence'][0], mode='binary')[:10]

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

Create a transformation pipleline to prepare the training dataset for RNN.

In [19]:
# This class selects the desired attributes and drops the rest.
class DataFrameSelector(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

In [20]:
# This class converts a nucleotide base (A, C, G, T) to one-hot-encoding.
class one_hot_encoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.tokenizer = Tokenizer(num_words=4, lower=False, char_level=True)

    def fit(self, X, y=None):
        # Note that X is a data frame.
        # Fit the tokenizer on the 1st sequence in the dataset.
        self.tokenizer.fit_on_texts(X.iloc[0, 0])
        self.len_sequence = len(X.iloc[0, 0])
        return self

    def transform(self, X):
        # Note that X is a data frame.
        one_hot_X = X.iloc[:, 0].map(lambda x: tokenizer.texts_to_matrix(x, mode='binary')).values
        one_hot_X = np.concatenate(one_hot_X)
        one_hot_X = np.reshape(one_hot_X, (-1, self.len_sequence, 4))
        return one_hot_X

In [21]:
attrbs = ['sequence']
my_pipeline = Pipeline([
    ('selector', DataFrameSelector(attrbs)),
    ('one_hot_encoder', one_hot_encoder())
])

In [22]:
X = my_pipeline.fit_transform(all_data_df)
X.shape

(2968, 200, 4)

Check the 1st sequence is correctly encoded.

In [23]:
X[0, :10, :]

array([[0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]])

In [24]:
y = all_data_df['enhancer'].values
y.shape

(2968,)

In [25]:
y[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Grid search over a range of network sizes

In [26]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)

In [29]:
# Create model; we set max 800 epochs based on the early training session.
# model = KerasClassifier(build_fn=create_model, epochs=800, batch_size=128, verbose=0)
model = KerasClassifier(build_fn=create_model, epochs=750, batch_size=128, verbose=0)

In [30]:
param_grid = {'recurrent1': [8, 16], 'recurrent2': [0, 8, 16], 'dense1': [0, 8, 16]}

In [31]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)  # random_state only used when shuffle == True

In [32]:
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=skf)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=skf)

In [33]:
t1 = datetime.datetime.now()
print('Training start time = %s' %  t1)
grid_result = grid.fit(X, y)
print('\nGrid Search Duration = %s' % (datetime.datetime.now()-t1))

Training start time = 2019-02-17 09:13:17.300745


model params : 8 0 0



model params : 8 0 0



model params : 8 0 0



model params : 8 0 0



...

model params : 16 16 16



model params : 16 16 16



model params : 16 16 16



model params : 8 0 0




Grid Search Duration = 4:06:35.530452


In [34]:
cv_results = pd.DataFrame.from_dict(grid_result.cv_results_, orient='columns')
cv_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 23 columns):
mean_fit_time         18 non-null float64
std_fit_time          18 non-null float64
mean_score_time       18 non-null float64
std_score_time        18 non-null float64
param_dense1          18 non-null object
param_recurrent1      18 non-null object
param_recurrent2      18 non-null object
params                18 non-null object
split0_test_score     18 non-null float64
split1_test_score     18 non-null float64
split2_test_score     18 non-null float64
split3_test_score     18 non-null float64
split4_test_score     18 non-null float64
mean_test_score       18 non-null float64
std_test_score        18 non-null float64
rank_test_score       18 non-null int32
split0_train_score    18 non-null float64
split1_train_score    18 non-null float64
split2_train_score    18 non-null float64
split3_train_score    18 non-null float64
split4_train_score    18 non-null float64
mean_train_score     

/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_a

In [41]:
# cv_results.iloc[:, [0,4,5,6,7,8,14,15,16,22,23]]
cv_results.iloc[:, [5, 6, 4, 0, 13, 14, 15, 21]].sort_values('rank_test_score')

,param_recurrent1,param_recurrent2,param_dense1,mean_fit_time,mean_test_score,std_test_score,rank_test_score,mean_train_score
0,8,0,0,101.301898,0.754043,0.013654,1,0.772997
1,8,8,0,170.501071,0.751685,0.009385,2,0.787061
12,8,0,16,126.055408,0.751348,0.017952,3,0.779398
3,16,0,0,105.667629,0.751011,0.017936,4,0.790179
13,8,8,16,189.496978,0.750674,0.011868,5,0.788159
7,8,8,8,179.792138,0.750000,0.009165,6,0.798098
6,8,0,8,112.252614,0.747642,0.020697,7,0.772742
15,16,0,16,131.846739,0.741577,0.016619,8,0.809468
9,16,0,8,120.048750,0.739218,0.016348,9,0.800033
10,16,8,8,184.752519,0.733491,0.013588,10,0.818568


In [40]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.754043 using {'dense1': 0, 'recurrent1': 8, 'recurrent2': 0}
